# Customized Image Classification using AutoGluon

In this tutorial, we load images and the corresponding labels into [AutoGluon](https://autogluon.mxnet.io/index.html) and use this data to obtain a neural network that can classify new images. This is different from traditional machine learning where we need to manually define the neural network and then specify the hyperparameters in the training process. Instead, with just a single call to AutoGluon’s fit function, AutoGluon automatically trains many models with different hyperparameter configurations and returns the model that achieved the highest level of accuracy.

Note: Please use **GPU** for training. CPU training will lead to an unceasing running script. 

In [1]:
! pip install -q nvidia-ml-py3==7.352.0
! pip install -q torch==1.8.0
! pip install -q torchvision==0.9.0
! pip install -q d2l==0.16.0
! pip install -q numpy==1.19.5
! pip install -q setuptools==54.1.1
! pip install -q wheel==0.36.2
! pip install -q autogluon==0.1.0
! pip install -q timm==0.4.12

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.2.8 requires dill>=0.3.4, but you have dill 0.3.3 which is incompatible.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.3 which is incompatible.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.23.25 which is incompatible.


Let's import the ImagePredictor

In [2]:
from autogluon.vision import ImagePredictor

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.6.0` and `torch==1.8.0` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


To use AutoGluon for computer vision task training, we need to organize our data with the following structure:

    data/
    ├── train/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...
    ├── test/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...

Here each subfolder contains all images that belong to that category, e.g., `class1` contains all images belonging to the first class. We generally recommend at least 100 training images per class for reasonable classification performance, but this might depend on the type of images in your specific use-case.

## Download the dataset

For demonstration purposes, we use a subset of the __Shopee-IET__ dataset from [Kaggle](https://www.kaggle.com/competitions). Each image in this data depicts a clothing item and the corresponding label specifies its clothing category. Our subset of the data contains the following possible labels: BabyPants, BabyShirt, womencasualshoes, womenchiffontop.

We download the data subset and create training/test dataset folders like below. If you use this on your own dataset, just point it to your training or test folder. Example: `train_dataset = ImagePredictor.Dataset.from_folder('mydataset/train')`

In [3]:
path = 'https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip'
train_dataset, _, test_dataset = ImagePredictor.Dataset.from_folders(path)

100%|██████████| 40895/40895 [00:02<00:00, 18951.01KB/s]


data/
├── test/
└── train/


Let's print the training dataset.

In [4]:
print(train_dataset)

                                                 image  label
0    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
1    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
2    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
3    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
4    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
..                                                 ...    ...
795  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
796  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
797  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
798  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
799  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3

[800 rows x 2 columns]


## Use AutoGluon to Fit Models

Now, let's fit a __classifier__ using AutoGluon [predictor.fit()](https://auto.gluon.ai/stable/tutorials/image_prediction/beginner.html). Within fit, the dataset is __automatically__ split into training and validation sets. The model with the best hyperparameter configuration is selected based on its performance on the __validation set__.

In [5]:
predictor = ImagePredictor()

time_limit = 10 * 60 # how long fit() should run (in seconds)
predictor.fit(train_dataset,
              epochs=10,
              time_limit=time_limit,
              ngpus_per_trial=1
             )

INFO:gluoncv.auto.tasks.image_classification:Randomly split train_data into train[728]/validation[72] splits.
INFO:gluoncv.auto.tasks.image_classification:Starting fit without HPO
INFO:TorchImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:TorchImageClassificationEstimator:root.train.epochs    200 != 15
INFO:TorchImageClassificationEstimator:root.train.batch_size 32 != 16
INFO:TorchImageClassificationEstimator:root.misc.seed       42 != 564
INFO:TorchImageClassificationEstimator:root.img_cls.model   resnet101 != resnet50
INFO:TorchImageClassificationEstimator:}
INFO:TorchImageClassificationEstimator:Saved config to /home/ec2-user/SageMaker/MLA-CV-Course/notebooks/day_2/960700cc/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50_ram-a26f946b.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth
INFO:TorchImageClassificationEstimator:Model resnet50 created, param count:  

## Model Results

Autogluon also provides the training results, which can be accessed by calling `predictor.fit_summary()`. 

In [6]:
fit_result = predictor.fit_summary()

In [7]:
fit_result

{'train_acc': 0.8541666666666666,
 'valid_acc': 0.9722222222222222,
 'total_time': 270.5811548233032,
 'best_config': {'model': 'resnet50',
  'lr': 0.01,
  'num_trials': 1,
  'epochs': 15,
  'batch_size': 16,
  'nthreads_per_trial': 128,
  'ngpus_per_trial': 1,
  'time_limits': 600,
  'search_strategy': 'random',
  'dist_ip_addrs': None,
  'log_dir': '/home/ec2-user/SageMaker/MLA-CV-Course/notebooks/day_2/960700cc',
  'num_workers': 4,
  'gpus': [0],
  'seed': 564,
  'final_fit': False,
  'wall_clock_tick': 1643329357.5302048,
  'problem_type': 'multiclass'},
 'fit_history': {'train_acc': 0.8541666666666666,
  'valid_acc': 0.9722222222222222,
  'total_time': 270.5811548233032,
  'best_config': {'model': 'resnet50',
   'lr': 0.01,
   'num_trials': 1,
   'epochs': 15,
   'batch_size': 16,
   'nthreads_per_trial': 128,
   'ngpus_per_trial': 1,
   'time_limits': 600,
   'search_strategy': 'random',
   'dist_ip_addrs': None,
   'log_dir': '/home/ec2-user/SageMaker/MLA-CV-Course/notebooks/da

We can access certain results from this summary. For example, training and validation accuracies below.

In [8]:
print('Train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Train acc: 0.854, val acc: 0.972


The best model and optimum hyperparameters: Learning rate, batch size, epochs can be printed with this:

In [9]:
fit_result['fit_history']['best_config']

{'model': 'resnet50',
 'lr': 0.01,
 'num_trials': 1,
 'epochs': 15,
 'batch_size': 16,
 'nthreads_per_trial': 128,
 'ngpus_per_trial': 1,
 'time_limits': 600,
 'search_strategy': 'random',
 'dist_ip_addrs': None,
 'log_dir': '/home/ec2-user/SageMaker/MLA-CV-Course/notebooks/day_2/960700cc',
 'num_workers': 4,
 'gpus': [0],
 'seed': 564,
 'final_fit': False,
 'wall_clock_tick': 1643329357.5302048,
 'problem_type': 'multiclass'}

## Making Predictions

We can call the predict function to run on different images.

In [10]:
image_path = test_dataset.iloc[0]['image']
predictor.predict(image_path)

,class,score,id
0,BabyPants,0.881436,0


Let's get predictions on the test set.

In [11]:
pred = predictor.predict(test_dataset)
print(pred)

               class     score  id  \
0          BabyPants  0.881436   0   
1          BabyPants  0.505561   0   
2   womencasualshoes  0.488053   2   
3          BabyPants  0.409595   0   
4          BabyPants  0.906190   0   
..               ...       ...  ..   
75   womenchiffontop  0.875525   3   
76   womenchiffontop  0.794147   3   
77   womenchiffontop  0.924260   3   
78   womenchiffontop  0.835401   3   
79   womenchiffontop  0.762412   3   

                                                image  
0   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
1   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
2   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
3   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
4   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
..                                                ...  
75  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
76  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
77  /home/ec2-user/.gluoncv/datasets/sh